# Genshin Impact: Assistant
### Name: Andrew Angulo

----------

# Preface: 

If you've been diving into the world of Genshin Impact, you probably know how exciting and challenging
it can be. Some folks find it like a dream come true, while for others, it's more like a quest. But what's that
quest all about? Well, think about any game you've played – there's always those newcomers who want to
rock the leaderboard, right? In Genshin Impact, it's no different. There are spots in the game where you
can test how strong your squad is. And that's where my Python script comes into play – it's designed to
give you a hand in reaching those goals.
See, Genshin Impact is all about building a team that packs a punch. You've got these cool areas where
you can measure just how tough your characters are. To make them even tougher, players spend time
getting better artifacts – those are like power-ups for your characters. With my Python script, the aim is to
help you figure out which character should be your go-to powerhouse, your "main carry," as you make
your mark in the game. So, let's team up and tackle those Genshin Impact challenges together!


# Goals:
- Be able to view their loadout without logging into the game
- Determine which artifacts work best for any character
- Create teams to allow them to prioritize their damage
- Give them basic information on a character
- Alert them of upcoming events
- Compare characters with other players
- Rate their artifacts
---




### Description of Imports:

- we use **pandas** so once we grab our data and store it into a Dictionary of List we can then transfer that data into a Dataframe


- we use **requests** so we can grab all of the HTML from just the URL


- we use **lmxl import html** because since we are working with **requests** we can utilize xpath to parse and traverse the html contents and return the data we need


- we use **os.path** in order to obtain the file based on the location specificed within the computer


- we use **json** so we are able to load data from a JSON files such as creds.json


- we use **sqlalchemy** so we are able to interact with the SQL database

In [160]:
%pip install enkanetwork.py


import pandas as pd
import requests
import os.path
import json
import sqlalchemy as sa
import asyncio
import pprint
from lxml import html
from enkanetwork import EnkaNetworkAPI
from enkanetwork import EquipmentsType, DigitType

Note: you may need to restart the kernel to use updated packages.


### Utilizing EnkaNetwork API

In [161]:
# Afif UID: 607566990
# Pheu UID: 701473745
# ---------------------------------------------------- #
UID = 701473745
client = EnkaNetworkAPI()
data = await client.fetch_user(UID)

async def main():
    async with client:
        print("=== Player Info ===")
        print(f"Nickname: {data.player.nickname}")
        print(f"Level: {data.player.level}")
        print(f"Icon: {data.player.avatar.icon.url}")
        print(f"Signature: {data.player.signature}")
        print(f"Achievement: {data.player.achievement}")
        print(f"Abyss floor: {data.player.abyss_floor} - {data.player.abyss_room}")
        print(f"Cache timeout: {data.ttl}")

await main()

=== Player Info ===
Nickname: Pheubell
Level: 60
Icon: https://enka.network/ui/UI_AvatarIcon_Beidou.png
Signature: i will seggs beidou and cyno
Achievement: 1044
Abyss floor: 12 - 3
Cache timeout: 50


### Fetching Character Data

In [162]:
players_Characters = {'Characters':[], 'Level':[], 'Element':[], }
async def characters():
    async with client:
        for char in data.characters:
            players_Characters['Characters'].append(char.name)
            players_Characters['Level'].append(char.level)
            players_Characters['Element'].append(char.element[0:10])

            
await characters()

print(players_Characters)

{'Characters': ['Yae Miko', 'Yelan', 'Dehya', 'Raiden Shogun', 'Hu Tao', 'Eula', 'Nahida', 'Kamisato Ayaka'], 'Level': [90, 90, 90, 90, 90, 90, 90, 90], 'Element': ['Electric', 'Water', 'Fire', 'Electric', 'Fire', 'Ice', 'Grass', 'Ice']}


### Fetching Artifact Data

In [292]:
Artifacts = {'Artifact_Name':[], 'Artifact_Level':[] , 'MainStatName':[], 'MainStatVal': [], 'SubStatName':[], 'SubStatVal': []}
artfList, artfLevel, artfStatName, artfMStatVal, artfSStatName, artfSStatVal = [], [], [], [], [], []

async def artifact(): #Fixed but could use some optimization
    atfNamePH, atfLevelPH, atfStatNamePH, artfMStatValPH, artfSStatNamePH, artfSStatValPH = [], [], [], [], [], []


    async with client:
        for char in data.characters:   
            for artf in filter(lambda x: x.type == EquipmentsType.ARTIFACT, char.equipments):
                atfNamePH.append(artf.detail.name) #Artifact Name
                atfLevelPH.append(artf.level) #Artifact level
                atfStatNamePH.append(artf.detail.mainstats.name) #Main Stat name
                artfMStatValPH.append(artf.detail.mainstats.value) #Main Stat Values
                #print(artfMStatValPH) #debugging
                for substat in artf.detail.substats:
                    #print(artfSStatNamePH)
                    artfSStatNamePH.append(substat.name) #substat name
                    artfSStatValPH.append(substat.value) #substat value



        for i in range(len(atfNamePH)):


            artfList.append(atfNamePH[i])
            artfLevel.append(atfLevelPH[i])
            artfStatName.append(atfStatNamePH[i])
            artfMStatVal.append(artfMStatValPH[i])
            


            #print(artfList)  #Debug
            if(len(artfList) == 5):
                #print(artfList) #Debug
                Artifacts['Artifact_Name'].append(artfList.copy()) #Copies Artifact name
                Artifacts['Artifact_Level'].append(artfLevel.copy()) #Copies Artifact Level
                Artifacts['MainStatName'].append(artfStatName.copy()) #Copies Main Stat Name
                Artifacts['MainStatVal'].append(artfMStatVal.copy()) #Copies Main Stat Values

                artfList.clear()
                artfLevel.clear()
                artfStatName.clear()
                artfMStatVal.clear()

        for z in range(len(artfSStatNamePH)):


            artfSStatName.append(artfSStatNamePH[z])
            artfSStatVal.append(artfSStatValPH[z])


            if(len(artfSStatName) == 4): #Each substat has 4 names
                Artifacts['SubStatName'].append(artfSStatName.copy())
                Artifacts['SubStatVal'].append(artfSStatVal.copy())


                artfSStatName.clear()
                artfSStatVal.clear()

                
                
                
await artifact()
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(Artifacts['Artifact_Name'])

[ [ "Gladiator's Nostalgia",
    "Gladiator's Destiny",
    'Hourglass of Thunder',
    'Scarlet Vessel',
    "Thunder Summoner's Crown"],
  [ 'Magnificent Tsuba',
    'Sundered Feather',
    'Storm Cage',
    'Goblet of Thundering Deep',
    'Ornate Kabuto'],
  [ 'Magnificent Tsuba',
    'Sundered Feather',
    'Storm Cage',
    'Pearl Cage',
    'Ornate Kabuto'],
  [ 'Magnificent Tsuba',
    'Sundered Feather',
    "Lavawalker's Torment",
    'Scarlet Vessel',
    'Ornate Kabuto'],
  [ "Witch's Flower of Blaze",
    "Witch's Ever-Burning Plume",
    "Witch's End Time",
    "Witch's Heart Flames",
    "Maiden's Fading Beauty"],
  [ 'Stainless Bloom',
    "Wise Doctor's Pinion",
    'Moment of Cessation',
    'Lamp of the Lost',
    'Mocking Mask'],
  [ 'Labyrinth Wayfarer',
    'Scholar of Vines',
    'A Time of Insight',
    'Chalice of the Font',
    'Laurel Coronet'],
  [ 'Snowswept Memory',
    "Icebreaker's Resolve",
    "Frozen Homeland's Demise",
    'Omen of Thunderstorm',
   

### Information comes together


In [293]:
StatSummarized = {}

for i in players_Characters['Characters']: #Characters Names
    StatSummarized[i] = {} #{'Yae Miko': {}, 'Yelan': {}, 'Dehya': {}, 'Raiden Shogun': {}, 'Hu Tao': {}, 'Eula': {}, 'Nahida': {}, 'Kamisato Ayaka': {}}# Ex: {'Yae Miko': {'Artifact': {}}

for char, artifactName in zip(StatSummarized, Artifacts['Artifact_Name']):  # Ex: {'Yae Miko': {'Artifact': {}}
    for artifact in artifactName[:5]:  # Take the first 5 artifacts 
        StatSummarized[char][artifact] = {}


for char, artifactName, MainStatName in zip(StatSummarized, Artifacts['Artifact_Name'], Artifacts['MainStatName']):
    StatSummarized[char][artifactName[0]][MainStatName[0]] = []
    StatSummarized[char][artifactName[1]][MainStatName[1]] = []
    for artifact, MSN in zip(artifactName[2:5], MainStatName[2:5]):  # Take the second : 5 artifacts 
        StatSummarized[char][artifact][MSN] = []

# [  "Gladiator's Nostalgia",
#       "Gladiator's Destiny",
#       'Hourglass of Thunder',
#       'Scarlet Vessel',
#       "Thunder Summoner's Crown"]

for char, artifactName, MainStatName, SubStatNames, SubStatVals in zip(StatSummarized, Artifacts['Artifact_Name'], Artifacts['MainStatName'], Artifacts['SubStatName'], Artifacts['SubStatVal']):             
    for artifact, MSN in zip(artifactName, MainStatName):
        print("-")
        for SSN, SSV in zip(SubStatNames, SubStatVals): # Fix Bug for later keeps on repeating here and not adding the other substat names/values
            print(SSN, SSV)
            StatSummarized[char][artifact][MSN].append({SSN : SSV})          
                        
#pp.pprint(StatSummarized)                        
#print("Yae Miko:",StatSummarized['Yae Miko'])
#print("Kamisato Ayaka:",StatSummarized['Kamisato Ayaka'])
pp.pprint(StatSummarized['Yae Miko'])

-
DEF 19
CRIT DMG 24.9
Elemental Mastery 23
CRIT Rate 6.6
-
DEF 19
CRIT DMG 24.9
Elemental Mastery 23
CRIT Rate 6.6
-
DEF 19
CRIT DMG 24.9
Elemental Mastery 23
CRIT Rate 6.6
-
DEF 19
CRIT DMG 24.9
Elemental Mastery 23
CRIT Rate 6.6
-
DEF 19
CRIT DMG 24.9
Elemental Mastery 23
CRIT Rate 6.6
-
CRIT Rate 17.9
CRIT DMG 7.8
ATK 9.9
DEF 23
-
CRIT Rate 17.9
CRIT DMG 7.8
ATK 9.9
DEF 23
-
CRIT Rate 17.9
CRIT DMG 7.8
ATK 9.9
DEF 23
-
CRIT Rate 17.9
CRIT DMG 7.8
ATK 9.9
DEF 23
-
CRIT Rate 17.9
CRIT DMG 7.8
ATK 9.9
DEF 23
-
ATK 4.1
CRIT DMG 20.2
HP 15.7
CRIT Rate 7.8
-
ATK 4.1
CRIT DMG 20.2
HP 15.7
CRIT Rate 7.8
-
ATK 4.1
CRIT DMG 20.2
HP 15.7
CRIT Rate 7.8
-
ATK 4.1
CRIT DMG 20.2
HP 15.7
CRIT Rate 7.8
-
ATK 4.1
CRIT DMG 20.2
HP 15.7
CRIT Rate 7.8
-
CRIT Rate 6.6
HP 9.9
CRIT DMG 19.4
Elemental Mastery 44
-
CRIT Rate 6.6
HP 9.9
CRIT DMG 19.4
Elemental Mastery 44
-
CRIT Rate 6.6
HP 9.9
CRIT DMG 19.4
Elemental Mastery 44
-
CRIT Rate 6.6
HP 9.9
CRIT DMG 19.4
Elemental Mastery 44
-
CRIT Rate 6.6
HP 9.9
